In [1]:
import keras_hub
import pathlib
import random

import keras
from keras import ops

import tensorflow.data as tf_data
import tensorflow.strings as tf_string

c:\adriana\official-code\2025-ASL-data\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from pyparsing import Word, alphas, nums
import pyparsing as pp
import re

In [3]:
handshapelist = [
  '1',
  '3',
  '4',
  '5',
  '6',
  '7',
  '8',
  '9',
  '10',
  '25',
  'A',
  'B',
  'C',
  'D',
  'E',
  'F',
  'G',
  'H-U',
  'I',
  'K',
  'L',
  'M',
  'N',
  'O',
  'R',
  'S',
  'T',
  'U',
  'V',
  'W',
  'X',
  'Y',
  'C-L',
  'U-L',
  'B-L',
  'P-K',
  'Q-G',
  'L-X',
  'I-L-Y',
  '5-C',
  '5-C-L',
  '5-C-tt',
  'alt-M',
  'alt-N',
  'alt-P',
  'B-xd',
  'baby-O',
  'bent-1',
  'bent-B',
  'bent-B-L',
  'bent-horns',
  'bent-L',
  'bent-M',
  'bent-N',
  'bent-U',
  'cocked-8',
  'cocked-F',
  'cocked-S',
  'crvd-5',
  'crvd-B',
  'crvd-flat-B',
  'crvd-L',
  'crvd-sprd-B',
  'crvd-U',
  'crvd-V',
  'fanned-flat-O',
  'flat-B',
  'flat-C',
  'flat-F',
  'flat-G',
  'flat-O-2',
  'flat-O',
  'full-M',
  'horns',
  'loose-E',
  'O-2-horns',
  'open-8',
  'open-F',
  'sml-C-3',
  'tight-C-2',
  'tight-C',
  'X-over-thumb'
]

In [4]:
number_regexp = r"""
    [0-9]+                      # one or more digits
    (?:\.[0-9]+)?               # optional decimal
    s?                          # optional trailing 's'
"""

alpha_regexp = r"""
    (?!                         # negative lookahead for:
        (?:THUMB-)?             # optional THUMB-
        (?:IX-|POSS-|SELF-)     # followed by IX-, POSS-, SELF-
    )
    [A-Z0-9]                    # starts with uppercase or digit
    (?:                         # optionally followed by:
        [A-Z0-9'-]*             #   more letters, digits, hyphen or apostrophe
        [A-Z0-9]                #   ends on a letter or digit
    )?
    (?:                         # optionally followed by:
        \.                      #   a period
      |                         # or
        :[0-9]                  #   colon-digit (e.g., :2)
    )?
"""

lookahead_regexp = r"""
    (?:
        (?![a-z])               # not followed by lowercase letter
      | (?=wg)                  # unless it's specifically "wg"
    )
"""

word_all_regexp = r"""
    (?: {number_regexp} | {alpha_regexp} )
    {lookalookahead_regexphead}
"""

In [5]:
cl_prefix = pp.one_of(["CL", "DCL", "LCL", "SCL", "BCL", "BPCL", "PCL", "ICL"])
ns_prefix = pp.Literal("ns")
fs_prefix = pp.Literal("fs")
lex_exceptions = pp.one_of(["part", "WHAT"])
aspect_text = pp.Literal("aspect")
index_core_ix = pp.Literal("IX")
other_index_core = pp.one_of(["POSS", "SELF"])
handshape = pp.one_of(handshapelist)
person = pp.one_of(["1p", "2p", "3p"])
dash = pp.Literal("-")
arc = pp.Literal("arc") 
loc = pp.Literal("loc")
pl = pp.Literal("pl")
compound = pp.Literal("+")
hashtag = pp.Literal("#")
choice = pp.Literal("/")
sym = pp.Literal(">")
par1 = pp.Literal("(")
par2 = pp.Literal(")")
contraction = pp.Literal("^")
colon = pp.Literal(":")
omit_quote = pp.Literal("xx")
period = pp.Literal(".")
alphas = pp.Word(alphas, max=1)
nums = pp.Word(nums, max=1)
word = pp.Regex(word_all_regexp, flags=re.X)

In [6]:
full_grammar = pp.OneOrMore(
    cl_prefix |               # classifiers like CL, DCL, etc.
    ns_prefix |               # non-specific ns
    fs_prefix |               # fingerspelling fs
    index_core_ix |           # IX
    other_index_core |        # POSS, SELF
    person |                  # 1p, 2p, 3p
    lex_exceptions |          # part, WHAT
    aspect_text |             # aspect
    arc |                     # arc
    loc |                     # loc
    pl |                      # pl
    handshape |               # handshapes like B, 1, 5, etc.
    compound |                # +
    hashtag |                 # #
    choice |                  # /
    sym |                     # >
    contraction |             # ^
    colon |                   # :
    dash |                    # -
    par1 | par2 |             # ( and )
    omit_quote |              # xx
    period |                  # .
    word |
    nums |                    # numbers last resort
    alphas                     # fallback LAST
)

In [7]:
trial = full_grammar.parse_string("SCL:1xx", parse_all=True).asList()
trial2 = full_grammar.parse_string("IX-1p-pl-2 WORK LANDSCAPE fs-LANDSCAPING IX-1p 5xx", parse_all=True).as_list()

print(trial)
print(trial2)

['SCL', ':', '1', 'xx']
['IX', '-', '1p', '-', 'pl', '-', '2', 'W', 'O', 'R', 'K', 'L', 'A', 'N', 'D', 'S', 'C', 'A', 'P', 'E', 'fs', '-', 'L', 'A', 'N', 'D', 'S', 'C', 'A', 'P', 'I', 'N', 'G', 'IX', '-', '1p', '5', 'xx']


In [8]:
def custom_tokenize(text):
    try:
        return full_grammar.parse_string(text, parse_all=True).as_list()
    except pp.ParseException as pe:
        print(f"Failed to parse: {pe}")
        return []

In [9]:
trial = custom_tokenize("SCL:1xx")
trial2 = custom_tokenize("IX-1p-pl-2 WORK LANDSCAPE fs-LANDSCAPING IX-1p 5xx")

print(trial)
print(trial2)

['SCL', ':', '1', 'xx']
['IX', '-', '1p', '-', 'pl', '-', '2', 'W', 'O', 'R', 'K', 'L', 'A', 'N', 'D', 'S', 'C', 'A', 'P', 'E', 'fs', '-', 'L', 'A', 'N', 'D', 'S', 'C', 'A', 'P', 'I', 'N', 'G', 'IX', '-', '1p', '5', 'xx']


In [10]:
BATCH_SIZE = 64
EPOCHS = 30  # This should be at least 10 for convergence
MAX_SEQUENCE_LENGTH = 50
ENG_VOCAB_SIZE = 5056
ASL_VOCAB_SIZE = 2283
num_samples = 1400

EMBED_DIM = 128
INTERMEDIATE_DIM =1024
NUM_HEADS = 8
data_path = "C:\\adriana\\official-code\\2025-ASL-data\\data\\sent_pairs.csv"

In [11]:
input_texts = []
target_texts = []
token_list = []
input_tokens = set()
target_tokens = set()
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
    
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split("\t")
    tokenized_sent = custom_tokenize(target_text)
    token_list.append(tokenized_sent)
    input_texts.append(input_text)
    target_texts.append(target_text)
    for token in input_text:
        if token not in input_tokens:
            input_tokens.add(token)
            
            
for sent in token_list:
    for tok in sent:
        if tok not in target_tokens:
            target_tokens.add(tok)


input_tokens = sorted(list(input_tokens))
target_tokens = sorted(list(target_tokens))

print("input_tokens:", input_tokens)
print("output_tokens", target_tokens)
num_encoder_tokens = len(input_tokens)
num_decoder_tokens = len(target_tokens)
print("num_eng_tokens", num_encoder_tokens)
print("num_asl_tokens", num_decoder_tokens)
print(token_list)

ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
parsed_sent = []
for sent in token_list:
    parsed_sent.append(" ".join(sent))

print(parsed_sent)

In [ ]:
def normalize_and_split(x):
    "Split on whitespace."
    return tf_string.split(x)

In [ ]:
def train_word_piece(text_samples, vocab_size, reserved_tokens):
    word_piece_ds = tf_data.Dataset.from_tensor_slices(text_samples)
    vocab = keras_hub.tokenizers.compute_word_piece_vocabulary(
        word_piece_ds.batch(1000).prefetch(2),
        vocabulary_size=vocab_size,
        reserved_tokens=reserved_tokens,
    )
    return vocab

In [ ]:
reserved_tokens = ["[PAD]", "[UNK]", "[START]", "[END]"]

eng_samples = [text_pair[0] for text_pair in train_pairs]
eng_vocab = train_word_piece(eng_samples, ENG_VOCAB_SIZE, reserved_tokens)

spa_samples = [text_pair[1] for text_pair in train_pairs]
spa_vocab = train_word_piece(spa_samples, SPA_VOCAB_SIZE, reserved_tokens)